In [ ]:
import pandas as pd

custom = pd.read_excel('./Phenotyping/custom_phenotypes.xlsx')
custom['phecode1.2'] = custom['phecode1.2'].astype(float)

icd = pd.read_fwf('./Phenotyping/icd10cm-order-2025.txt', header=None)
icd = icd[[1,3]].set_axis(['ICD','Description'],axis=1)
icd['Category'] = icd['ICD'].str[0]
icd['Number'] = icd['ICD'].str[1:3].str.extract('(\d+)').astype(int)
icd['Level'] = icd['ICD'].str.len()

icd = icd.loc[icd['Level'] == 3]
icd = icd.loc[~icd['Category'].isin(['A','B','C','O','P','S','T','U','V','W','X','Y','Z'])]
icd = icd.loc[~((icd['Category'] == 'D') & (icd['Number'] <= 53))]
# Communicable, maternal, perinatal and nutritional conditions
# https://platform.who.int/mortality/about/list-of-causes-and-corresponding-icd-10-codes
pattern = r'^(A[0-9]{2}|B[0-9]{2}|D5[0-3]|D64|E0[0-2]|E4[0-6]|E5[0-9]|E6[0-4]|G0[0-4]|G14|H6[5-6]|J[0-2][0-9]|N7[0-3]|O[0-9]{2}|P[0-9]{2}|U04|U07|U09|U10)$'
icd = icd.loc[~icd['ICD'].str.match(pattern)]
icd.reset_index(drop=True).to_csv('./Phenotyping/icd_codes.csv', index=False)

temp = custom.loc[~custom['single_icd'].isin(icd['ICD'])]
temp = temp[['custom_code','icd_category','phenotype']].drop_duplicates().set_axis(['Code','Category','Description'],axis=1)
temp_icd = icd[['ICD','Category','Description']].set_axis(['Code','Category','Description'],axis=1)
temp = pd.concat([temp_icd,temp]).reset_index(drop=True)
temp.to_excel('./Phenotyping/merged_phenotypes.xlsx', index=False)